<center><h1>Time Series Classification(1)</h1></center>
<br>
<br>

Name: Chengyi Li
<br>
Github Username: ChengyiLi98
<br>

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [1]:
import os
import csv
import numpy as np
import pandas as pd
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats

Get the AReM Data Set

In [2]:
main_path = '../data/ArEM/'
file_list = os.listdir(main_path)
file_list

['bending1', 'bending2', 'cycling', 'lying', 'sitting', 'standing', 'walking']

In [3]:
values = [7,6,15,15,15,15,15]
folders = dict(zip(file_list, values))

paths =[]
for folder in folders:
    for num in range(1, folders[folder] + 1):
        path = main_path + folder + '/dataset' + str(num) + '.csv'
        paths.append(path)
# paths will be the list of the paths of all the datasets

In [4]:
#paths

### (b) Test and Train Data

In [5]:
test_path =[]
test_paths =[]

for folder in file_list[:2]:
    for num in range(1,3):
        test_path = main_path + folder + '/dataset' + str(num) + '.csv'
        test_paths.append(test_path)

for folder in file_list[2:]:
    for num in range(1,4):
        test_path = main_path + folder + '/dataset' + str(num) + '.csv'
        test_paths.append(test_path)

test_paths

['../data/ArEM/bending1/dataset1.csv',
 '../data/ArEM/bending1/dataset2.csv',
 '../data/ArEM/bending2/dataset1.csv',
 '../data/ArEM/bending2/dataset2.csv',
 '../data/ArEM/cycling/dataset1.csv',
 '../data/ArEM/cycling/dataset2.csv',
 '../data/ArEM/cycling/dataset3.csv',
 '../data/ArEM/lying/dataset1.csv',
 '../data/ArEM/lying/dataset2.csv',
 '../data/ArEM/lying/dataset3.csv',
 '../data/ArEM/sitting/dataset1.csv',
 '../data/ArEM/sitting/dataset2.csv',
 '../data/ArEM/sitting/dataset3.csv',
 '../data/ArEM/standing/dataset1.csv',
 '../data/ArEM/standing/dataset2.csv',
 '../data/ArEM/standing/dataset3.csv',
 '../data/ArEM/walking/dataset1.csv',
 '../data/ArEM/walking/dataset2.csv',
 '../data/ArEM/walking/dataset3.csv']

In [6]:
train_paths =[]
for i in range(len(paths)):
    if paths[i] not in test_paths:
        train_paths.append(paths[i])       

In [7]:
'''
# minor cleaning
DF = []
for item in paths:
    try:
        df = pd.read_csv(item, skiprows=4, on_bad_lines = 'error')
        DF.append(df)
    except pd.errors.ParserError as e:
        print(f'Error while reading {item}: {e}')
        
'''        
#Error while reading ../data/ArEM/cycling/dataset9.csv: Error tokenizing data. C error: Expected 7 fields in line 485, saw 8

#Error while reading ../data/ArEM/cycling/dataset14.csv: Error tokenizing data. C error: Expected 7 fields in line 485, saw 8

"\n# minor cleaning\nDF = []\nfor item in paths:\n    try:\n        df = pd.read_csv(item, skiprows=4, on_bad_lines = 'error')\n        DF.append(df)\n    except pd.errors.ParserError as e:\n        print(f'Error while reading {item}: {e}')\n        \n"

In [8]:
# change the delimiter of this dataset and manully modify the first four lines
'''
input_file = '../data/ArEM/bending2/dataset4.csv'
new_data = []

with open(input_file, 'r', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        new_row = [cell.replace(' ', ',') for cell in row]
        new_data.append(new_row)

with open(input_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(new_data)  

output_file = input_file

with open(input_file, 'r') as f_in:
    lines = f_in.readlines()

with open(output_file, 'w') as f_out:
    for line in lines:
        update_line = line.strip('"\n')
        update_line = update_line[:-1]
        f_out.write(update_line + '\n')
'''

'\ninput_file = \'../data/ArEM/bending2/dataset4.csv\'\nnew_data = []\n\nwith open(input_file, \'r\', newline=\'\') as f:\n    reader = csv.reader(f)\n    for row in reader:\n        new_row = [cell.replace(\' \', \',\') for cell in row]\n        new_data.append(new_row)\n\nwith open(input_file, \'w\', newline=\'\') as f:\n    writer = csv.writer(f)\n    writer.writerows(new_data)  \n\noutput_file = input_file\n\nwith open(input_file, \'r\') as f_in:\n    lines = f_in.readlines()\n\nwith open(output_file, \'w\') as f_out:\n    for line in lines:\n        update_line = line.strip(\'"\n\')\n        update_line = update_line[:-1]\n        f_out.write(update_line + \'\n\')\n'

In [9]:
train_set = []
test_set = []
DF_test = []
DF_train = []

for item in test_paths:
    df = pd.read_csv(item, skiprows=4)
    DF_test.append(df)

for item in train_paths:
    df = pd.read_csv(item, skiprows=4)
    DF_train.append(df)
else:
    df = pd.read_csv(item, skiprows=4)
            
test_set = pd.concat(DF_test, ignore_index=True)
train_set = pd.concat(DF_train, ignore_index=True)


In [10]:
test_set

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,0,39.25,0.43,22.75,0.43,33.75,1.30
1,250,39.25,0.43,23.00,0.00,33.00,0.00
2,500,39.25,0.43,23.25,0.43,33.00,0.00
3,750,39.50,0.50,23.00,0.71,33.00,0.00
4,1000,39.50,0.50,24.00,0.00,33.00,0.00
...,...,...,...,...,...,...,...
9115,118750,36.00,2.45,17.00,5.10,20.50,0.87
9116,119000,34.33,1.89,15.00,2.45,17.00,2.12
9117,119250,33.00,7.35,14.60,3.14,13.00,5.70
9118,119500,31.67,1.25,11.00,6.16,19.25,2.17


In [11]:
train_set

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,0,42.00,0.71,21.25,0.43,30.00,0.00
1,250,41.50,0.50,20.25,1.48,31.25,1.09
2,500,41.50,0.50,14.25,1.92,33.00,0.00
3,750,40.75,0.83,15.75,0.43,33.00,0.00
4,1000,40.00,0.71,20.00,2.74,32.75,0.43
...,...,...,...,...,...,...,...
33114,118750,34.50,6.18,9.00,3.56,12.67,4.19
33115,119000,25.75,6.02,13.75,2.05,16.00,1.58
33116,119250,31.50,3.35,10.25,5.12,16.25,2.95
33117,119500,33.75,2.77,14.00,3.24,13.75,0.43


### (c) Feature Extraction

#### i. Research

minimum, maximum, mean, median, range, IQR, percentile, std, variance, root mean square, entropy ...

#### ii. Extraction

In [12]:
table = pd.DataFrame()

for item in paths:
    df = pd.read_csv(item, skiprows=4)
    k = df.describe()
    df_temp = k.T[1:]
    df_temp = df_temp.rename(columns={'25%':'1st quart','50%':'median','75%':'3rd quart'})
    ordered_col = ['min','max','mean','median','std','1st quart','3rd quart']
    df_temp = df_temp[ordered_col]
    row_list = []
    for i in range(6):
        row = ','.join(map(str, df_temp.iloc[i].values))
        row_list.append(',' + row)  # Add a comma before each row
    single_row = ' '.join(row_list)
    single_row = single_row[1:]
    df_now = pd.DataFrame([single_row.split(',')])
    table = pd.concat([table, df_now], axis=0)

In [13]:
ordered_col = ['min','max','mean','median','std','1st quart','3rd quart']
new_col =[]
for i in range(1, 7):
    for item in ordered_col:
        new_col.append(f'{item}{i}')
table.columns = new_col
table.index = range(1, 89)
table.index.name = 'Instance'
table

,min1,max1,mean1,median1,std1,1st quart1,3rd quart1,min2,max2,mean2,...,std5,1st quart5,3rd quart5,min6,max6,mean6,median6,std6,1st quart6,3rd quart6
Instance,,,,,,,,,,,,,,,,,,,,,
1,37.25,45.0,40.62479166666667,40.5,1.4769669162668118,39.25,42.0,0.0,1.3,0.35860416666666667,...,2.1884486069244624,33.0,36.0,0.0,1.92,0.5705833333333333,0.43,0.5829154940538757,0.0,1.3
2,38.0,45.67,42.8128125,42.5,1.4355499139997019,42.0,43.67,0.0,1.22,0.3724375,...,1.9952545563428268,32.0,34.5,0.0,3.11,0.5710833333333334,0.43,0.6010096527048443,0.0,1.3
3,35.0,47.4,43.9545,44.33,1.558835066353028,43.0,45.0,0.0,1.7,0.4262500000000001,...,1.9996040626699223,35.3625,36.5,0.0,1.79,0.49329166666666674,0.43,0.5135061012349963,0.0,0.94
4,33.0,47.75,42.1798125,43.5,3.670665660998427,39.150000000000006,45.0,0.0,3.0,0.6960416666666666,...,3.8494478189457717,30.4575,36.33,0.0,2.18,0.6135208333333334,0.5,0.5243171317817953,0.0,1.0
5,33.0,45.75,41.6780625,41.75,2.2434900014223933,41.33,42.75,0.0,2.83,0.5359791666666667,...,2.4110263914491896,28.4575,31.25,0.0,1.79,0.3832916666666667,0.43,0.38916429696825316,0.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,19.5,45.33,33.586875,34.25,4.650935459926406,30.25,37.0,0.0,14.67,4.5765625,...,3.2839834287518306,13.73,18.25,0.0,8.32,3.259729166666667,3.11,1.6402432891439511,2.05,4.3225
85,19.75,45.5,34.32275,35.25,4.752477031316844,31.0,38.0,0.0,13.47,4.456333333333333,...,3.1198562340300238,13.5,17.75,0.0,9.67,3.4325625000000004,3.2,1.7327271926847894,2.1574999999999998,4.5649999999999995
86,19.5,46.0,34.54622916666666,35.25,4.84229350816215,31.25,37.8125,0.0,12.47,4.371958333333334,...,2.8231240558264434,14.0,17.75,0.0,10.0,3.3381250000000002,3.08,1.6567423530609375,2.16,4.335


#### iii. Standard Deviation

In [14]:
table = table.astype(float)
S = table.std()
# Estimate the standard deviation of each of the time-domain features

In [15]:
temp = pd.DataFrame()
df_list = []

for i in range(len(S)):
    feature_name = S.index[i]
    std_value = S[i]
    sample = table.iloc[:, i].values

    # build a 90% bootstrap confidence interval
    result = bs.bootstrap(sample, stat_func=bs_stats.std, alpha = 0.1)

    # Extract the lower and upper bounds of the confidence interval
    lower_ci = result.lower_bound
    upper_ci = result.upper_bound
    temp = pd.DataFrame({'Feature': [feature_name], 'Std': [std_value], 'Lower_CI': [lower_ci], 'Upper_CI': [upper_ci]})

    df_list.append(temp)

results_df = pd.concat(df_list, ignore_index=True)
results_df

,Feature,Std,Lower_CI,Upper_CI
0,min1,9.569975,8.322444,10.792761
1,max1,4.394362,3.476285,5.437408
2,mean1,5.335718,4.768743,5.926901
3,median1,5.440054,4.854866,6.059633
4,std1,1.772153,1.587037,1.962307
5,1st quart1,6.153590,5.625687,6.694548
6,3rd quart1,5.138925,4.411216,5.910477
7,min2,0.000000,0.000000,0.000000
8,max2,5.062729,4.688440,5.467701
9,mean2,1.574164,1.433076,1.741110


#### iv. Select Features

In [16]:
range_df = results_df['Feature'].copy()
re = results_df['Upper_CI'] - results_df['Lower_CI']

In [17]:
DF_range = pd.concat([range_df, re], axis=1)
DF_range = DF_range.rename(columns={0: 'Range'})
DF_range.sort_values('Range').head(10)

,Feature,Range
21,min4,0.000000
7,min2,0.000000
25,std4,0.065047
39,std6,0.066503
35,min6,0.078029
26,1st quart4,0.116872
40,1st quart6,0.117627
11,std2,0.136746
23,mean4,0.146220
24,median4,0.148049


min, 1st quart, std. Because for the top 10 lowest range, they appear more frequently.

The range of the confidence intervals show the stability. The more narrow the interval is, which means the range smaller; the more stabilty and degree of precision the model will have.